<a href="https://colab.research.google.com/github/pavanashriofficial/practice-codes/blob/main/granite_3_2_2b_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers accelerate torch pillow gradio huggingface_hub

import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from PIL import Image
import torch
import re

print("Loading IBM Granite 3.2 2B Instruct model...")

# Load model and tokenizer
model_name = "ibm-granite/granite-3.2-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

# Load image-to-text model for image analysis
print("Loading image analysis model...")
image_to_text = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")

print("Models loaded successfully!")

def generate_response(prompt, max_length=512):
    """Generate response using IBM Granite model"""
    messages = [{"role": "user", "content": prompt}]
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response.strip()

def chatbot_function(user_message, history):
    """Main chatbot function"""
    if not user_message:
        return history

    bot_response = generate_response(user_message)
    history.append((user_message, bot_response))
    return history

def verify_prescription(prescription_text):
    """Advanced prescription verification with dosage and safety checks"""
    if not prescription_text:
        return "⚠️ Please enter prescription details to verify."

    prompt = f"""You are an expert pharmaceutical AI system. Conduct a comprehensive analysis of this prescription:

{prescription_text}

Provide a detailed report with these sections:

1. **PRESCRIPTION COMPLETENESS CHECK**
   - Patient information verification
   - Prescriber details validation
   - Date and signature verification

2. **MEDICATION DOSAGE ANALYSIS**
   - Review each medication dosage
   - Identify any unusual or potentially dangerous dosages
   - Flag if dosages exceed standard therapeutic ranges
   - Check frequency and duration appropriateness

3. **SAFETY ALERTS**
   - Potential drug interactions
   - Contraindications
   - Age-specific warnings
   - Pregnancy/breastfeeding warnings if applicable

4. **VERIFICATION STATUS**
   - Overall assessment: APPROVED / NEEDS REVIEW / REJECTED
   - Critical issues (if any)
   - Recommendations

Use ⚠️ for warnings, ✓ for approved items, and ❌ for critical issues."""

    verification_result = generate_response(prompt, max_length=1000)
    return f"## 📋 Prescription Verification Report\n\n{verification_result}"

def medicine_finder(medicine_name):
    """Look up detailed drug information"""
    if not medicine_name:
        return "⚠️ Please enter a medicine name to search."

    prompt = f"""You are a pharmaceutical database expert. Provide comprehensive information about: {medicine_name}

Include:

**💊 DRUG INFORMATION**
- Generic Name
- Brand Names
- Drug Class/Category
- Mechanism of Action

**📝 INDICATIONS**
- Primary uses
- Approved conditions

**⚕️ DOSAGE INFORMATION**
- Standard adult dosage
- Pediatric dosage (if applicable)
- Maximum daily dose
- Administration guidelines

**⚠️ SIDE EFFECTS**
Common side effects:
- List 5-7 most common

Serious side effects:
- List critical adverse reactions
- When to seek immediate medical attention

**🚫 CONTRAINDICATIONS**
- Who should not take this medication
- Medical conditions to avoid

**🔄 DRUG INTERACTIONS**
- Major interactions to avoid
- Foods/supplements that interact

**💡 PATIENT ADVICE**
- Best time to take
- With or without food
- Storage instructions
- What to avoid while taking

Provide accurate, evidence-based information."""

    drug_info = generate_response(prompt, max_length=1200)
    return f"## 🔍 Medicine Information: {medicine_name}\n\n{drug_info}"

def detect_side_effects(medicine_list):
    """Analyze side effects for multiple medications"""
    if not medicine_list:
        return "⚠️ Please enter medication names (one per line)."

    prompt = f"""You are a clinical pharmacology expert. Analyze the side effect profile for these medications:

{medicine_list}

Provide:

**⚠️ INDIVIDUAL SIDE EFFECTS**
For each medication, list:
- Common side effects (>10% occurrence)
- Uncommon but significant side effects
- Rare but serious adverse reactions

**🔴 COMBINED RISK ANALYSIS**
- Overlapping side effects when taken together
- Cumulative risks
- Synergistic adverse effects

**📊 SEVERITY CLASSIFICATION**
- Mild (manageable, no intervention needed)
- Moderate (may require medical attention)
- Severe (requires immediate medical intervention)

**💡 PATIENT MONITORING**
- What symptoms to watch for
- When to contact healthcare provider
- Emergency warning signs

**🛡️ RISK MITIGATION**
- How to minimize side effects
- Timing considerations
- Lifestyle modifications

Use color coding: 🟢 Mild | 🟡 Moderate | 🔴 Severe"""

    side_effects_analysis = generate_response(prompt, max_length=1200)
    return f"## ⚕️ Side Effects Analysis\n\n{side_effects_analysis}"

def analyze_image(image):
    """Analyze uploaded medical images or prescriptions"""
    if image is None:
        return "⚠️ Please upload an image to analyze."

    caption_result = image_to_text(image)
    caption = caption_result[0]['generated_text'] if caption_result else "Unable to generate caption"

    prompt = f"""You are a medical imaging and prescription analysis expert. Based on this image description: "{caption}"

Provide:

**🔍 IMAGE ANALYSIS**
- Type of document/image detected
- Quality and readability assessment
- Key visible elements

**📋 CONTENT EXTRACTION**
If this is a prescription or medical document:
- Patient information (if visible)
- Medication names and dosages
- Prescriber information
- Date and validity

**✓ VERIFICATION NOTES**
- Completeness check
- Legibility concerns
- Missing information
- Recommendations for clarification

**⚠️ SAFETY CONSIDERATIONS**
- Any unusual observations
- Potential concerns
- Suggested follow-up actions"""

    detailed_analysis = generate_response(prompt, max_length=800)

    result = f"## 📸 Image Analysis Report\n\n**Detected Content:** {caption}\n\n{detailed_analysis}"
    return result

# Premium CSS styling
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Playfair+Display:wght@400;700&family=Inter:wght@300;400;600&display=swap');

:root {
    --gold: #D4AF37;
    --dark-gold: #B8941E;
    --black: #0A0A0A;
    --white: #FFFFFF;
    --light-gray: #F5F5F5;
    --dark-gray: #1A1A1A;
}

body {
    background: linear-gradient(135deg, #0A0A0A 0%, #1A1A1A 100%);
    font-family: 'Inter', sans-serif;
}

.gradio-container {
    max-width: 1400px !important;
    margin: 0 auto !important;
}

/* Header Styling */
h1, h2, h3 {
    font-family: 'Playfair Display', serif !important;
    color: var(--gold) !important;
    text-shadow: 0 2px 10px rgba(212, 175, 55, 0.3);
}

h1 {
    font-size: 3em !important;
    text-align: center;
    margin: 20px 0 !important;
    letter-spacing: 2px;
}

/* Tab Styling */
.tabs {
    background: var(--dark-gray);
    border-radius: 15px;
    padding: 10px;
    border: 2px solid var(--gold);
}

.tab-nav button {
    background: var(--black) !important;
    color: var(--white) !important;
    border: 1px solid var(--gold) !important;
    border-radius: 10px !important;
    margin: 5px !important;
    padding: 12px 25px !important;
    font-weight: 600 !important;
    transition: all 0.3s ease !important;
}

.tab-nav button:hover {
    background: var(--gold) !important;
    color: var(--black) !important;
    transform: translateY(-2px);
    box-shadow: 0 5px 15px rgba(212, 175, 55, 0.4);
}

.tab-nav button.selected {
    background: linear-gradient(135deg, var(--gold) 0%, var(--dark-gold) 100%) !important;
    color: var(--black) !important;
    font-weight: 700 !important;
    box-shadow: 0 5px 20px rgba(212, 175, 55, 0.5);
}

/* Button Styling */
.btn-primary {
    background: linear-gradient(135deg, var(--gold) 0%, var(--dark-gold) 100%) !important;
    color: var(--black) !important;
    border: none !important;
    border-radius: 10px !important;
    padding: 12px 30px !important;
    font-weight: 700 !important;
    font-size: 1em !important;
    transition: all 0.3s ease !important;
    box-shadow: 0 4px 15px rgba(212, 175, 55, 0.3);
}

.btn-primary:hover {
    transform: translateY(-3px);
    box-shadow: 0 6px 25px rgba(212, 175, 55, 0.5) !important;
}

/* Input Fields */
textarea, input {
    background: var(--dark-gray) !important;
    color: var(--white) !important;
    border: 2px solid var(--gold) !important;
    border-radius: 10px !important;
    padding: 15px !important;
    font-size: 1em !important;
}

textarea:focus, input:focus {
    border-color: var(--gold) !important;
    box-shadow: 0 0 20px rgba(212, 175, 55, 0.3) !important;
}

/* Output Areas */
.output-markdown, .output-text {
    background: var(--dark-gray) !important;
    color: var(--light-gray) !important;
    border: 2px solid var(--gold) !important;
    border-radius: 10px !important;
    padding: 20px !important;
    min-height: 200px;
}

/* Chatbot Styling */
.message-wrap {
    background: var(--dark-gray) !important;
    border-radius: 10px !important;
    padding: 15px !important;
    margin: 10px 0 !important;
    border: 1px solid var(--gold) !important;
}

.message.user {
    background: linear-gradient(135deg, var(--gold) 0%, var(--dark-gold) 100%) !important;
    color: var(--black) !important;
}

.message.bot {
    background: var(--dark-gray) !important;
    color: var(--white) !important;
    border: 1px solid var(--gold) !important;
}

/* Labels */
label {
    color: var(--gold) !important;
    font-weight: 600 !important;
    font-size: 1.1em !important;
    margin-bottom: 10px !important;
}

/* Image Upload */
.image-container {
    border: 3px dashed var(--gold) !important;
    border-radius: 15px !important;
    background: var(--dark-gray) !important;
    padding: 20px !important;
}

/* Scrollbar */
::-webkit-scrollbar {
    width: 12px;
}

::-webkit-scrollbar-track {
    background: var(--black);
}

::-webkit-scrollbar-thumb {
    background: var(--gold);
    border-radius: 6px;
}

::-webkit-scrollbar-thumb:hover {
    background: var(--dark-gold);
}

/* Footer */
.footer {
    text-align: center;
    color: var(--gold);
    padding: 20px;
    font-size: 0.9em;
    border-top: 2px solid var(--gold);
    margin-top: 30px;
}
"""

# Create Premium Gradio Interface
with gr.Blocks(css=custom_css, title="AI Medical Intelligence Hub") as demo:
    gr.Markdown("# ⚕️ AI MEDICAL INTELLIGENCE HUB")
    gr.Markdown("### *Premium Medical Analysis & Prescription Intelligence System*")
    gr.Markdown("---")

    with gr.Tabs():
        # Tab 1: AI Assistant
        with gr.Tab("🤖 AI Medical Assistant"):
            gr.Markdown("### Intelligent Medical Consultation")
            chatbot_interface = gr.Chatbot(
                height=500,
                label="Conversation",
                avatar_images=("👤", "⚕️")
            )
            msg = gr.Textbox(
                label="Your Query",
                placeholder="Ask me anything about medications, health concerns, or medical information...",
                lines=3
            )
            with gr.Row():
                submit_btn = gr.Button("Send", variant="primary", size="lg")
                clear_btn = gr.ClearButton([msg, chatbot_interface], value="Clear Chat", size="lg")

            submit_btn.click(chatbot_function, inputs=[msg, chatbot_interface], outputs=[chatbot_interface])
            msg.submit(chatbot_function, inputs=[msg, chatbot_interface], outputs=[chatbot_interface])

        # Tab 2: Advanced Prescription Verification
        with gr.Tab("📋 Prescription Verification"):
            gr.Markdown("### Advanced Prescription Analysis & Dosage Verification")
            gr.Markdown("*Comprehensive safety checks including unusual dosage detection*")

            prescription_input = gr.Textbox(
                label="Prescription Details",
                placeholder="""Enter complete prescription information:

Patient Name: John Doe
Age: 45 | Gender: Male
Doctor: Dr. Sarah Smith, MD
Date: 2024-11-17

Medications:
1. Amoxicillin 500mg - Take 3 times daily for 7 days
2. Ibuprofen 400mg - Take as needed for pain (max 3 times daily)
3. Omeprazole 20mg - Take once daily before breakfast""",
                lines=12
            )
            verify_btn = gr.Button("🔍 Verify Prescription", variant="primary", size="lg")
            prescription_output = gr.Markdown(label="Verification Report")

            verify_btn.click(verify_prescription, inputs=[prescription_input], outputs=[prescription_output])

        # Tab 3: Medicine Finder
        with gr.Tab("💊 Medicine Finder"):
            gr.Markdown("### Comprehensive Drug Information Lookup")
            gr.Markdown("*Search detailed information about any medication*")

            medicine_input = gr.Textbox(
                label="Medicine Name",
                placeholder="Enter medication name (e.g., Aspirin, Metformin, Lisinopril)...",
                lines=2
            )
            search_btn = gr.Button("🔍 Search Drug Information", variant="primary", size="lg")
            medicine_output = gr.Markdown(label="Drug Information")

            search_btn.click(medicine_finder, inputs=[medicine_input], outputs=[medicine_output])

        # Tab 4: Side Effects Detector
        with gr.Tab("⚠️ Side Effects Analysis"):
            gr.Markdown("### Advanced Side Effects Detection & Risk Analysis")
            gr.Markdown("*Analyze side effects and drug interactions for multiple medications*")

            medicine_list_input = gr.Textbox(
                label="Medication List",
                placeholder="""Enter medications (one per line):

Aspirin 81mg
Metformin 500mg
Lisinopril 10mg
Atorvastatin 20mg""",
                lines=10
            )
            analyze_btn = gr.Button("⚕️ Analyze Side Effects", variant="primary", size="lg")
            side_effects_output = gr.Markdown(label="Side Effects Report")

            analyze_btn.click(detect_side_effects, inputs=[medicine_list_input], outputs=[side_effects_output])

        # Tab 5: Image Analysis
        with gr.Tab("📸 Prescription Image Analysis"):
            gr.Markdown("### Medical Image & Prescription OCR Analysis")
            gr.Markdown("*Upload prescription images for AI-powered analysis*")

            with gr.Row():
                with gr.Column(scale=1):
                    image_input = gr.Image(type="pil", label="Upload Prescription/Medical Image")
                    analyze_img_btn = gr.Button("🔍 Analyze Image", variant="primary", size="lg")

                with gr.Column(scale=1):
                    image_output = gr.Markdown(label="Analysis Results")

            analyze_img_btn.click(analyze_image, inputs=[image_input], outputs=[image_output])

    gr.Markdown("---")
    gr.Markdown(
        """<div class='footer'>
        <p><strong>⚕️ AI Medical Intelligence Hub</strong> | Powered by IBM Granite 3.2 2B Instruct</p>
        <p style='font-size: 0.85em; color: #B8941E;'>⚠️ For informational purposes only. Always consult healthcare professionals for medical decisions.</p>
        </div>""",
        elem_classes="footer"
    )

# Launch the interface
print("\n✨ Launching AI Medical Intelligence Hub...")
print("🔐 Premium Edition - Black, White & Gold Theme")
demo.launch(share=True, debug=True)

Loading IBM Granite 3.2 2B Instruct model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


Models loaded successfully!


/tmp/ipython-input-3526032231.py:414: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_interface = gr.Chatbot(



✨ Launching AI Medical Intelligence Hub...
🔐 Premium Edition - Black, White & Gold Theme
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4b79f148672dde1153.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4b79f148672dde1153.gradio.live


## Local Inference on GPU
Model page: https://huggingface.co/ibm-granite/granite-3.2-2b-instruct

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/ibm-granite/granite-3.2-2b-instruct)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="ibm-granite/granite-3.2-2b-instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.2-2b-instruct")
model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-3.2-2b-instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))